In [1]:
# Copyright 2025 Quantinuum (www.quantinuum.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
import stim 
import time
from Code614 import *

In [3]:
def checks(shot):
    init_meas = 40 + 36 + 36 + 12
    aux_meas = 40 + 36 + 36
    resource_meas = 40
    #First we check that everything was initialized correctly
    for i in range(0,40):
        if (shot[i]):
            return False


    #Then we post-select on any non-trivial |++> resource state stabilizers
    for j in range(0,12):
        s0 = shot[resource_meas +6*j]^shot[resource_meas + 1 +6*j]^shot[resource_meas + 2 + 6*j]^shot[resource_meas + 3 + 6*j]
        s1 = shot[resource_meas + 2 +6*j]^shot[resource_meas + 3 +6*j]^shot[resource_meas + 4 + 6*j]^shot[resource_meas + 5 + 6*j]
        if (s0 or s1):
            return False

    
    #Then we compute what Y corrections to apply to the [[36,4,4]] + aux measurements based on the logical measured values for the resource states
    #Recall that a measurement value of 1 actually corresponds to |i+>, in which case no correction is needed for a Ry(-pi/2) rotation
    for j in range(0,6):
        l0 = shot[resource_meas + 6*j]^shot[resource_meas + 2 + 6*j]^shot[resource_meas + 4 + 6*j]
        l1 = shot[resource_meas + 6*j + 1]^shot[resource_meas + 3 + 6*j]^shot[resource_meas + 5 + 6*j]
        if l0 == False:
            for i in range(0,3):
                shot[init_meas + 2*i + 6*j] = not shot[init_meas + 2*i + 6*j]
        if l1 == False:
            for i in range(0,3):
                shot[init_meas + 2*i + 1 + 6*j] = not shot[init_meas + 1 + 2*i + 6*j]
        #Any of the j resource state corrections will flip the aux0 block
        if l0 == False:
            for i in range(0,3):
                shot[aux_meas + 2*i] = not shot[aux_meas + 2*i]
        if l1 == False:
            for i in range(0,3):
                shot[aux_meas + 2*i + 1] = not shot[aux_meas + 1 + 2*i]


    #Recall that a measurement value of 0 actually corresponds to |i->, in which case no correction is needed for a Ry(+pi/2) rotation
    for j in range(0,6):
        l0 = shot[resource_meas + 6*(j+6)]^shot[resource_meas + 2 + 6*(j+6)]^shot[resource_meas + 4 + 6*(j+6)]
        l1 = shot[resource_meas + 1 + 6*(j+6)]^shot[resource_meas + 3 + 6*(j+6)]^shot[resource_meas + 5 + 6*(j+6)]
        if l0 == True:
            for i in range(0,3):
                shot[init_meas + 2*i + 6*j] = not shot[init_meas + 2*i + 6*j]
        if l1 == True:
            for i in range(0,3):
                shot[init_meas + 2*i + 1 + 6*j] = not shot[init_meas + 1 + 2*i + 6*j]

    # Check if any of the H-check auxiliary block logicals or stabilizers are non-zero
    # Aux 0 logicals
    if (shot[aux_meas]^shot[aux_meas+2]^shot[aux_meas+4] or shot[aux_meas+1]^shot[aux_meas+3]^shot[aux_meas+5]):
        return False
    #Aux 1 logicals
    if (shot[aux_meas+6]^shot[aux_meas+6+2]^shot[aux_meas+6+4] or shot[aux_meas+6+1]^shot[aux_meas+6+3]^shot[aux_meas+6+5]):
        return False
    #Aux 0 stabilizers
    if (shot[aux_meas]^shot[aux_meas+1]^shot[aux_meas+2]^shot[aux_meas+3] or shot[aux_meas+2]^shot[aux_meas+3]^shot[aux_meas+4]^shot[aux_meas+5]):
        return False
    #Aux 1 stabilizers
    if (shot[aux_meas+6]^shot[aux_meas+6+1]^shot[aux_meas+6+2]^shot[aux_meas+6+3] or shot[aux_meas+6+2]^shot[aux_meas+6+3]^shot[aux_meas+6+4]^shot[aux_meas+6+5]):
        return False

    #Finally, we post-select on any non-trivial [[36,4,4]] stabilizers
    #Level 1 stabilizers
    for j in range(0,6):
        s0 = shot[init_meas + 6*j]^shot[init_meas + 6*j+1]^shot[init_meas + 6*j+2]^shot[init_meas + 6*j+3]
        s1 = shot[init_meas + 6*j+2]^shot[init_meas + 6*j+3]^shot[init_meas + 6*j+4]^shot[init_meas + 6*j+5]
        if (s0 or s1):
            return False
    #level2 stabilizers
    for i in range(0,2):
        s0 = False
        s1 = False
        #Sum over the ith logical operator for q = [0,1,2,3] for the 0th stab and q = [2,3,4,5] for the first stab
        for q in range(0,4):
            s0 = s0^shot[init_meas + 6*q + i]^shot[init_meas + 6*q + 2 + i]^shot[init_meas + 6*q + 4 + i]
        for q in range(2,6):
            s1 = s1^shot[init_meas + 6*q + i]^shot[init_meas + 6*q + 2 + i]^shot[init_meas + 6*q + 4 + i]
        if (s0 or s1):
            return False
    #We've passed all checks
    return True

def success(shot):
    init_meas = 40 + 36 + 36 + 12
    aux_meas = 40 + 36 + 36
    resource_meas = 40
    # We pass if all of the logical operators are 0, i.e., the distilled state is |++++>_L
    for i in range(0,2):
        for j in range(0,2):
            lij = False
            # p sums over block horizontally, and q vertically
            for p in range(0,3):
                for q in range(0,3):
                    lij = lij^shot[init_meas + 2*p + 2*6*q + i + 6*j]
            if (lij):
                return False
    return True
def simulate_protocol(p, num_samples):
    m = p/10
    c = stim.Circuit()
    b = []
    # Get the two level 1 magic states (4 measurements)
    c += get_dist_circ(0, 200, p, m)
    c += get_dist_circ(6,200+2, p, m)
    for i in range(0,6):
        block = Code6(6*i, p, m)
        b.append(block)

    #Initialize the other 4 [[6,2,2]] codeblocks in the [[36,4,4]] code + the two aux blocks (12 measurements)
    for i in range(2,8):
        c += get_init_circ(6*i, 200 + 2*i, p,m)
    aux0 = Code6(36, p, m)
    aux1 = Code6(42, p, m)

    #Initialize the 12 resource magic states used to perform 6 ry(-pi/2)_L rotations before the CZ followed by 6 ry(pi/2) rotations after the CZ (24 meas)
    resource_states = []
    for i in range(8,20):
        block = Code6(6*i, p, m)
        resource_states.append(block)
        c += get_dist_circ(6*i, 200 + 2*i)

    c += b[2].hadamard()
    c += b[2].cnot(b[3])
    c += b[4].hadamard()
    c += b[4].cnot(b[5])
    #Adding memory error to the two codeblocks not acted on by transversal gates in this depth 1
    qss = [b[0].qubits, b[1].qubits]
    c.append("Depolarize1", [q for qs in qss for q in qs], m)
    c += b[2].cnot(b[0])
    c += b[3].cnot(b[1])
    
    qss = [b[4].qubits, b[5].qubits]
    c.append("Depolarize1", [q for qs in qss for q in qs], m)
    c += b[0].cnot(b[4])
    c += b[1].cnot(b[5])
    
    qss = [b[2].qubits, b[3].qubits]
    c.append("Depolarize1", [q for qs in qss for q in qs], m)
    c += b[4].cnot(b[2])
    c += b[5].cnot(b[3])
    
    qss = [b[2].qubits, b[3].qubits]
    c.append("Depolarize1", [q for qs in qss for q in qs], m)

    #We rotate all logical qubits to |0> by performing Ry(-pi/2) rotations
    for i in range(0,6):
        c += b[i].rypi2(resource_states[i])


    #Now we can measure the Z value into the aux blocks
    c += aux0.hadamard()
    c += aux0.cnot(aux1)

    c += aux0.cz(b[0])
    c += aux1.cz(b[1])
    qss = [b[2].qubits, b[3].qubits, b[4].qubits, b[5].qubits]
    c.append("Depolarize1", [q for qs in qss for q in qs], m)
    
    c += aux0.cz(b[2])
    c += aux1.cz(b[3])
    qss = [b[0].qubits, b[1].qubits, b[4].qubits, b[5].qubits]
    c.append("Depolarize1", [q for qs in qss for q in qs], m)
    
    c += aux0.cz(b[4])
    c += aux1.cz(b[5])
    qss = [b[0].qubits, b[1].qubits, b[2].qubits, b[3].qubits]
    c.append("Depolarize1", [q for qs in qss for q in qs], m)

    c += aux0.cnot(aux1)
    c += aux0.hadamard()

    #We rotate all logical qubits back to |+> by performing Ry(+pi/2) rotations
    for i in range(0,6):
        c += b[i].rypi2(resource_states[i+6])




    c += aux0.measure()
    c += aux1.measure()

    for i in range(0,6):
        c += b[i].hadamard()
        c += b[i].measure()
    s = c.compile_sampler()
    
    total = 0
    fail = 0
    for i in range(0,1):
        l = s.sample(num_samples)
        for shot in l:
            if (checks(shot)):
                total += 1
                if(not success(shot)):
                    fail += 1
        l = None
    return (fail,total,p)

In [6]:
failures, totals, err_rate = simulate_protocol(.004, 100000)
print('number of magic state errors:', failures, 'number of accepts:', totals, 'physical error rate:', err_rate)
print('logical failure rate:', failures/totals)

number of magic state errors: 0 number of accepts: 19959 physical error rate: 0.004
logical failure rate: 0.0
